# Join walkthrough
In databases, when we have two tables that are related to each other by a common element, then our database is called a relational database. When all of our data is in a single file, that's called a flat file. 

Often, in data, we have one set of information stored in a table over here, and another set of information stored in a table over here. At the university, your student records are scattered in tables all over. Somewhere, there is a master student record, that has your name, birthdate, ID number, home address and other basic info. Then, over in the registrars office, we have the classes you took and the grades you received. Over here, we have the bursars office, which shows how much you owe in tuition and how much you've paid. If we wanted to get a single table together that showed how much you paid for each grade you got, we'd have to JOIN them together somehow. 

In Agate, like SQL, it's called a join. So let's do that. I've got two datasets that I want to join together and calculate a new number from something in both. 

Here is the question we're trying to answer: What Nebraska city has seen the largest growth in taxable sales since The Great Recession. 

In [1]:
import agate

Now we need to import our two tables.

In [22]:
taxes2014 = agate.Table.from_csv('../../Data/taxes2014.csv')
print(taxes2014)
print(len(taxes2014.rows))

|--------------------+---------------|
|  column_names      | column_types  |
|--------------------+---------------|
|  Join14            | Text          |
|  County14          | Text          |
|  City14            | Text          |
|  NetTaxableSales14 | Number        |
|  NebraskaSaleTax14 | Number        |
|--------------------+---------------|

463


In [23]:
taxes2008 = agate.Table.from_csv('../../Data/taxes2008.csv')
print(taxes2008)
print(len(taxes2008.rows))

|---------------------+---------------|
|  column_names       | column_types  |
|---------------------+---------------|
|  Join08             | Text          |
|  County08           | Text          |
|  City08             | Text          |
|  NetTaxableSales08  | Number        |
|  NebraskaSalesTax08 | Number        |
|---------------------+---------------|

471


Join syntax could not be easier. It's create a new table, and set it equal to the table you want to start with dot join and then it's the table you want to join to your starting table, and then the fields you're going to join on, starting with your original table and then your second table. In my case, I created fields in the dataset that merged the name of the city and county in case there were multiple towns named something. I named those fields Join14 and Join08 to know what they were and to keep them straight. 

In [16]:
taxes = taxes2014.join(taxes2008, 'Join14', 'Join08', inner=True)

In [24]:
print(taxes)
print(len(taxes.rows))

|---------------------+---------------|
|  column_names       | column_types  |
|---------------------+---------------|
|  Join14             | Text          |
|  County14           | Text          |
|  City14             | Text          |
|  NetTaxableSales14  | Number        |
|  NebraskaSaleTax14  | Number        |
|  County08           | Text          |
|  City08             | Text          |
|  NetTaxableSales08  | Number        |
|  NebraskaSalesTax08 | Number        |
|---------------------+---------------|

454


Note some cities dropped out. That'll be because of reporting problems or changes between the reports. If we were doing a story, we'd investigate and figure out what happened and if we could fix it.

But for purposes of this assignment, the rest is stuff you've done. We'll calculate a percent change, sort it and print it. 

In [18]:
change = taxes.compute([
    ('taxable_change', agate.PercentChange('NetTaxableSales14', 'NetTaxableSales08')),
    ('salestax_change', agate.PercentChange('NebraskaSaleTax14', 'NebraskaSalesTax08'))        
])

In [19]:
sorted_change = change.order_by('taxable_change', reverse=True)

In [20]:
for_printing = sorted_change.select(['City14', 'taxable_change'])

In [21]:
for_printing.print_table()

|------------------+---------------------------------|
|  City14          |                 taxable_change  |
|------------------+---------------------------------|
|  Trumbull        | 2,506.958490415494783874892785  |
|  Crofton         |   363.440505829239578554811776  |
|  Smithfield      |   361.429315899185143073716127  |
|  Giltner         |   326.770738115438298804017625  |
|  Wilsonville     |   284.347101535531484347101536  |
|  Staplehurst     |   244.187309865275966970882225  |
|  Prosser         |   237.599810836864984812382910  |
|  Ayr             |   206.453196990048673397172032  |
|  Wellfleet       |   185.673168065424505130316260  |
|  Jansen          |   174.726964494639574031183592  |
|  Mead            |   122.536865829879671710618990  |
|  Winnebago       |   115.322053973804086367821780  |
|  Odell           |   108.086177540136577326155623  |
|  Craig           |   104.392029716788697313894316  |
|  Brewster        |   104.270002156566745740780677  |
|  Danbury

## Assignment for Tuesday

There are three data files in the Data folder in the class repository: frl13, frl14, and frl15. They are the Free and Reduced Lunch participation totals for every school in Nebraska. I want you to join them together into a single table and calculate the percent change from 2013 to 2015 and sort them by the largest. Which school in Nebraska saw the largest increase in participation in free and reduced school lunches, which is a proxy for poverty. **Hint: Duplicate field names are not allowed in Agate. Each file has the same field names. How could you fix that quickly?**